<a href="https://colab.research.google.com/github/rahiakela/deep-learning-research-and-practice/blob/main/deep-learning-with-python-by-francois-chollet/7-deep-dive-into-keras/02_keras_training_and_evaluation_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Keras model fundamentals

There are three APIs for building models in Keras:

* The Sequential model, the most approachable API—it’s basically a Python list. As such, it’s limited to simple stacks of layers.
* The Functional API, which focuses on graph-like model architectures. It represents
a nice mid-point between usability and flexibility, and as such, it’s the
most commonly used model-building API.
* Model subclassing, a low-level option where you write everything yourself from
scratch. This is ideal if you want full control over every little thing. However, you
won’t get access to many built-in Keras features, and you will be more at risk of
making mistakes.

<img src='https://github.com/rahiakela/deep-learning-research-and-practice/blob/main/deep-learning-with-python-by-francois-chollet/7-deep-dive-into-keras/images/1.png?raw=1' width='600'/>

##Setup

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.datasets import mnist

import random
import string
import re

import numpy as np

##Using built-in training and evaluation loops

The principle of progressive disclosure of complexity—access to a spectrum of workflows
that go from dead easy to arbitrarily flexible, one step at a time—also applies to
model training. Keras provides you with different workflows for training models. 

They
can be as simple as calling `fit()` on your data, or as advanced as writing a new training
algorithm from scratch.



In [2]:
def get_mnist_model():
  inputs = keras.Input(shape=(28 * 28, ))
  features = layers.Dense(512, activation="relu")(inputs)
  features = layers.Dropout(0.5)(features)
  outputs = layers.Dense(10, activation="softmax")(features)

  model = keras.Model(inputs=inputs, outputs=outputs)

  return model

Load your data, reserving
some for validation.

In [3]:
(images, labels), (test_images, test_labels) = mnist.load_data()

images = images.reshape((60000, 28 * 28)).astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28)).astype("float32") / 255

train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

11501568/11490434 [==============================] - 0s 0us/step


In [4]:
model = get_mnist_model()

# Compile the model by specifying its optimizer, the loss function to minimize, and the metrics to monitor
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# train the model, optionally providing validation data to monitor performance on unseen data
model.fit(train_images, train_labels, epochs=3, validation_data=(val_images, val_labels))

Epoch 1/3
1563/1563 [==============================] - 16s 10ms/step - loss: 0.2968 - accuracy: 0.9121 - val_loss: 0.1424 - val_accuracy: 0.9588
Epoch 2/3
1563/1563 [==============================] - 13s 8ms/step - loss: 0.1643 - accuracy: 0.9538 - val_loss: 0.1219 - val_accuracy: 0.9663
Epoch 3/3
1563/1563 [==============================] - 14s 9ms/step - loss: 0.1394 - accuracy: 0.9623 - val_loss: 0.1165 - val_accuracy: 0.9693


In [5]:
# compute the loss and metrics on new data
test_metrics = model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 3ms/step - loss: 0.1067 - accuracy: 0.9708


In [6]:
# compute classification probabilities on new data
predictions = model.predict(test_images)

There are a couple of ways you can customize this simple workflow:

* Provide your own custom metrics.
* Pass `callbacks` to the `fit()` method to schedule actions to be taken at specific points during training.


##Writing your own metrics

Metrics are key to measuring the performance of your model—in particular, to measuring
the difference between its performance on the training data and its performance
on the test data.

A Keras metric is a subclass of the `keras.metrics.Metric` class. Like layers, a metric
has an internal state stored in TensorFlow variables. Unlike layers, these variables
aren’t updated via backpropagation, so you have to write the state-update logic yourself,
which happens in the `update_state()` method.

For example, here’s a simple custom metric that measures the root mean squared
error (RMSE).

In [13]:
class RootMeanSquaredError(keras.metrics.Metric):

  def __init__(self, name="rmse", **kwargs):
    super().__init__(name=name, **kwargs)

    # Define the state variables in the constructor.
    self.mse_sum = self.add_weight(name="mse_sum", initializer="zeros")
    self.total_samples = self.add_weight(name="total_samples", initializer="zeros", dtype="int32")

  def update_state(self, y_true, y_pred, sample_weight=None):
    # To match our MNIST model, we expect categorical predictions and integer labels.
    y_true = tf.one_hot(y_true, depth=tf.shape(y_pred)[1])
    mse = tf.reduce_sum(tf.square(y_true - y_pred))
    self.mse_sum.assign_add(mse)
    num_samples = tf.shape(y_pred)[0]
    self.total_samples.assign_add(num_samples)

  def result(self):
    return tf.sqrt(self.mse_sum / tf.cast(self.total_samples, tf.float32))

  def reset_state(self):
    self.mse_sum.assign(0)
    self.total_samples.assign(0)

Now, Custom metrics can be used just like built-in ones. Let’s test-drive our own metric:

In [14]:
model = get_mnist_model()

# Compile the model by specifying its optimizer, the loss function to minimize, and the metrics to monitor
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy", RootMeanSquaredError()])

# train the model, optionally providing validation data to monitor performance on unseen data
model.fit(train_images, train_labels, epochs=3, validation_data=(val_images, val_labels))

Epoch 1/3
1563/1563 [==============================] - 15s 9ms/step - loss: 0.2943 - accuracy: 0.9125 - rmse: 7.1820 - val_loss: 0.1448 - val_accuracy: 0.9597 - val_rmse: 7.3612
Epoch 2/3
1563/1563 [==============================] - 16s 10ms/step - loss: 0.1661 - accuracy: 0.9535 - rmse: 7.3533 - val_loss: 0.1189 - val_accuracy: 0.9672 - val_rmse: 7.4027
Epoch 3/3
1563/1563 [==============================] - 13s 8ms/step - loss: 0.1386 - accuracy: 0.9632 - rmse: 7.3914 - val_loss: 0.1103 - val_accuracy: 0.9720 - val_rmse: 7.4186


In [15]:
# compute the loss and metrics on new data
test_metrics = model.evaluate(test_images, test_labels)

313/313 [==============================] - 1s 3ms/step - loss: 0.1014 - accuracy: 0.9730 - rmse: 7.4314


In [16]:
# compute classification probabilities on new data
predictions = model.predict(test_images)

##Using callbacks

This inputs object holds information about the shape and dtype of the data that the
model will process:

In [ ]:
inputs.shape

TensorShape([None, 3])

In [ ]:
inputs.dtype

tf.float32

We call such an object a symbolic tensor. It doesn’t contain any actual data, but it
encodes the specifications of the actual tensors of data that the model will see when
you use it. It stands for future tensors of data.

All Keras layers can be called both on real tensors of data and on these symbolic tensors.

In [ ]:
features.shape

TensorShape([None, 64])

In [ ]:
features.dtype

tf.float32

Here’s the summary of our model:

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 my_input (InputLayer)       [(None, 3)]               0         
                                                                 
 dense_4 (Dense)             (None, 64)                256       
                                                                 
 dense_5 (Dense)             (None, 10)                650       
                                                                 
Total params: 906
Trainable params: 906
Non-trainable params: 0
_________________________________________________________________


##Conclusion

In general, the Functional API provides you with a pretty good trade-off between
ease of use and flexibility. It also gives you direct access to layer connectivity, which is
very powerful for use cases such as model plotting or feature extraction. 

If you can use
the Functional API—that is, if your model can be expressed as a directed acyclic graph
of layers—I recommend using it over model subclassing.

In general, using Functional models
that include subclassed layers provides the best of both worlds: high development flexibility
while retaining the advantages of the Functional API.

